In [1]:
import torch
import numpy as np
import os
from tqdm import tqdm
from ultis.PointCloudDataLoader import PointCloudDataLoader
from feature_to_skeleton import makeSkeleton
from pointnet2_ssg_wo_normals.pointnet2_cls_ssg import get_model as get_model_ssg
from pointnet2_msg_normals.pointnet2_cls_msg import get_model as get_model_msg, get_loss

d:\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(1024,)
torch.Size([1024])
torch.Size([51])


In [27]:
from ultis.provider import *

In [2]:
path = 'data_196' #change path

In [3]:
data_train = PointCloudDataLoader(path, "train")
data_test = PointCloudDataLoader(root=path, split="test")

(10, 6, 196)
(10, 6, 196)


In [7]:
iterr = iter(trainDataLoader)
first = next(iterr)
points, target = first

In [8]:
print(target)

tensor([[[-2.9573e-01, -4.0345e-02,  1.8374e-04],
         [-4.2653e-01, -5.1735e-02,  2.9287e-02],
         [-4.0761e-01, -5.2567e-01,  1.3018e-01],
         [-3.8501e-01, -9.3739e-01,  2.4094e-01],
         [-1.6738e-01, -5.5608e-02,  7.8891e-03],
         [-2.2428e-01, -5.3188e-01,  1.2798e-01],
         [-2.4473e-01, -9.4196e-01,  2.3784e-01],
         [-2.9881e-01,  3.2493e-01, -6.1992e-02],
         [-3.0503e-01,  6.8523e-01, -1.5394e-01],
         [-3.1197e-01,  8.5582e-01, -2.6125e-01],
         [-3.1706e-01,  1.0123e+00, -2.4933e-01],
         [-1.0069e-01,  5.6681e-01, -8.8446e-02],
         [-5.8730e-02,  2.2390e-01,  9.0998e-02],
         [-6.1183e-02, -7.5171e-02,  1.6744e-01],
         [-4.9913e-01,  5.7636e-01, -1.2509e-01],
         [-5.5027e-01,  2.4450e-01,  3.2677e-02],
         [-5.4421e-01, -6.0775e-02,  6.8661e-02]],

        [[-2.9531e-01, -3.9960e-02,  2.0530e-04],
         [-4.2746e-01, -5.1544e-02,  2.9540e-02],
         [-4.0804e-01, -5.2537e-01,  1.3026e-01]

In [5]:
trainDataLoader = torch.utils.data.DataLoader(data_train, batch_size=2, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(data_test, batch_size=2, shuffle=False)

In [5]:
checkpoint = torch.load('D:\Documents\AIProject\HAR\pointnet2\Pointnet_Pointnet2_pytorch\log\classification\pointnet2_msg_normals\checkpoints\\best_model.pth')
model = get_model_msg(num_class=40)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model = model.cuda()

In [7]:
modelSkeleton = makeSkeleton()
modelSkeleton = modelSkeleton.cuda()

In [8]:
criterion = torch.nn.MSELoss()


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)
global_epoch = 0
global_step = 0
best_instance_acc = 0.0
best_class_acc = 0.0
start_epoch = 0 #checkpoint['epoch']

In [12]:
for epoch in range(start_epoch, 20):
    model = model.train()
    scheduler.step()
    for id, (points, target) in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader), smoothing=0.9):
        optimizer.zero_grad()
    

        points, target = points.cuda(), target.cuda()
        trans, features = model(points)
        features = torch.reshape(features, (features.shape[0], 1024))
        features = features.cuda()
        print(features.shape)
        pred_skeleton = modelSkeleton(features)
        print(pred_skeleton.shape)
        loss = criterion(torch.flatten(pred_skeleton), torch.flatten(target))
        print(f'loss is: {loss}')
        loss.backward()
        optimizer.step()
        global_step += 1

 20%|██        | 1/5 [00:00<00:01,  2.80it/s]

torch.Size([2, 1024])
torch.Size([2, 51])
loss is: 0.14180532097816467


 40%|████      | 2/5 [00:00<00:01,  2.44it/s]

torch.Size([2, 1024])
torch.Size([2, 51])
loss is: 0.14244703948497772


 60%|██████    | 3/5 [00:01<00:00,  2.44it/s]

...

torch.Size([2, 1024])
torch.Size([2, 51])
loss is: 0.14004822075366974


 80%|████████  | 4/5 [00:01<00:00,  2.59it/s]

torch.Size([2, 1024])
torch.Size([2, 51])
loss is: 0.1392030268907547


100%|██████████| 5/5 [00:01<00:00,  2.61it/s]

torch.Size([2, 1024])
torch.Size([2, 51])
loss is: 0.1398935765028


In [35]:
features2 = features2.numpy()


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [36]:
with numpy.printoptions(threshold=numpy.inf):
    print(features2)

[[[2.30094218e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [6.73071678e-41]
  [5.84113693e+00]
  [1.56850100e+00]
  [1.04252303e+00]
  [1.68080425e+00]
  [7.01652408e-01]
  [1.25898027e+00]
  [9.54514623e-01]
  [3.04808199e-01]
  [2.30113769e+00]
  [5.51075814e-03]
  [6.61314279e-02]
  [1.18993723e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [1.20927677e-01]
  [4.33171988e+00]
  [2.14684033e+00]
  [5.84814727e-01]
  [2.37565804e+00]
  [2.32604370e-02]
  [0.00000000e+00]
  [6.17519356e-02]
  [6.19148445e+00]
  [0.00000000e+00]
  [5.95185459e-01]
  [1.06949627e-01]
  [3.36730853e-02]
  [3.12295365e+00]
  [4.14347112e-01]
  [2.41903710e+00]
  [2.30450511e-01]
  [1.19443727e+00]
  [5.33529818e-01]
  [0.00000000e+00]
  [2.96249479e-01]
  [1.07648468e+00]
  [0.00000000e+00]
  [5.84472513e+00]
  [0.00000000e+00]
  [3.66736293e-01]
  [1.26659721e-01]
  [4.93276397e-32]
  [1.91259131e-01]
  [3.70894933e+00]
  [3.07175040e-01]
  [2.08965993e+00]
  [1.75951874e+00]
  [3.7475029

In [39]:
numpy.save("features_196/feature3d.npy", features2)
numpy.save("features_196/feature6d.npy", features1)

In [35]:
print(fake_features)

tensor([[[5.7356e-01],
         [0.0000e+00],
         [0.0000e+00],
         ...,
         [1.6945e+00],
         [7.5044e-01],
         [2.1510e-20]]])


In [39]:
new_feature = numpy.load('../features_196/feature.npy')
print(new_feature)

[[[2.3009422e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  ...
  [1.7057688e+00]
  [1.9821327e+00]
  [2.1510313e-20]]]
